In [1]:
from llama_index.core import Document, VectorStoreIndex, Settings, StorageContext
import json
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.llms import ChatMessage
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

In [3]:
llm = Ollama(model="llama3.2:3b", request_timeout=120.0)
Settings.llm = llm
embedding_model = OllamaEmbedding(
    model_name="mxbai-embed-large",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)
Settings.embed_model = embedding_model

In [4]:
with open("../data/processed/resolutions_processed.json") as f:
    data = json.load(f)

In [5]:
documents = [
    Document(
        text=item["full_text"],
        metadata={
            "name": item["name"],
            "resolution_date": item["resolution_date"],
            "concept": item["concept"],
            "process_date": item["process_date"],
        },
    )
    for item in data
]

In [7]:
db = chromadb.PersistentClient(path="../../../chroma_db")
chroma_collection = db.get_or_create_collection("upme_index")

In [8]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [9]:
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    async_insert=True,
    embedding_model=embedding_model
)

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("Dime sobre la RESOLUCION No. 000727 de 2024 de la UPME")
response